In [3]:
import psycopg2
engine = psycopg2.connect(
    database="betterreads",
    user="betterreadsadmin",
    password="betterreadsadmin",
    host="betterreads-datascience-database.cvmplnwee5ws.us-east-1.rds.amazonaws.com",
    port='5432')

In [4]:
import traceback
error_log = []
traces = []

def q(query, verbosity = False):
    data = 'No Data'
    curs = engine.cursor()
    try:
        curs.execute(query)
        if(verbosity):
            data = curs.fetchall()
    except psycopg2.errors.InvalidTextRepresentation as err:
        start = int(err.diag.statement_position)
        end = query.find('\'')
        replacement = '{'+query[start:end]+'}'
        new_query = query[:start] + replacement + query[start+len(replacement):]
        engine.rollback()
        q(new_query, verbosity)
        
    except Exception as e:
        engine.rollback()
        print(e)
        traceback.print_exc()
        error_log.append(e)

    engine.commit()
    curs.close()
    if(verbosity):
        return data

In [3]:
types_query = """
    select data_type 
    from information_schema.columns 
    where table_name = '{}';
    """
sql_to_py = {'text':str, 'ARRAY':list, 'integer':int}


#The data dict for each type holds the name given from the OpenLibrary data dump and the corresponding column name in the PostgreSQL database on AWS
edition_data_dict = {'title':'EditionTitle',
                    'authors':'Authors',
                    'genres':'Genre',
                    'subjects':'Subjects',
                    'publish_date':'PublishedYear',
                    'publishers':'Publisher',
                    'number_of_pages':'Pages',
                    'ratings':'Ratings',
                    'popularity':'Popularity',
                    'reviews':'Reviews',
                    'translated_from':'OriginalLanguage',
                    'location':'Nationality',
                    'translators':'Translators',
                    'work_titles':'WorkTitles',
                    'works':'Works',
                    'languages':'Languages',
                    'isbn_13':'ISBN13'}

editions_types = [sql_to_py[x[0]] for x in q(types_query.format('editions'), verbosity=True)]
editions_types_ol = {key:value for key,value in zip(list(edition_data_dict.keys()),editions_types)}
editions_types_db = {key:value for key,value in zip(list(edition_data_dict.values()),editions_types)}

# Author dict
author_data_dict = {'name':'Name',
                   'works':'Works',
                   'key':'Key'}

authors_types = [sql_to_py[x[0]] for x in q(types_query.format('authors'), verbosity=True)]
authors_types_ol = {key:value for key,value in zip(list(author_data_dict.keys()),authors_types)}
authors_types_db = {key:value for key,value in zip(list(author_data_dict.values()),authors_types)}

work_data_dict = {'title':'WorkTitle',
                 'authors':'Authors',
                 'subjects':'Subjects',
                 'description':'Description',
                 'first_publish_date':'FirstPublishDate',
                 'editions':'Editions',
                 'other_titles':'OtherTitles',
                 'translated_titles':'TranslatedTitles'}

works_types = [sql_to_py[x[0]] for x in q(types_query.format('works'), verbosity=True)]
works_types_ol = {key:value for key,value in zip(list(work_data_dict.keys()),works_types)}
works_types_db = {key:value for key,value in zip(list(work_data_dict.values()),works_types)}


In [4]:
#Helper method to generate dictionaries from their respective data_dict based on the type_line given
def generate_dictionaries(type_name):
    data_dict = globals()[('{}_data_dict'.format(type_name))]
    return {x: [] for x in data_dict.values()}

In [5]:
types_query = """
    select data_type 
    from information_schema.columns 
    where table_name = '{}';
    """

alpher = q(types_query.format('editions'), verbosity=True)
len(alpher)

18

In [6]:
#Helper method to verify the input and make sure it is formatted correctly, then returns it as a dictionary
def verify_input(input_string, type_name):
    try:
        input_string = input_string.replace('null', '')
        input_string = input_string.replace('true', 'True')
        input_string = input_string.replace('false', 'False')
        if(type(input_string) != dict):
            input_string = eval(input_string)
        
        data_dict = globals()[('{}_data_dict'.format(type_name))]
        types_dict = globals()[('{}s_types_ol'.format(type_name))]
        
        for key in data_dict.keys():
            if key not in input_string.keys():
                missing_key_type = types_dict[key]
                input_string[key] = missing_key_type ()
        
        return input_string
    
    except Exception as e:
        exceptions.append(('verify_input',traceback.format_exc(),e,input_string))

In [7]:
#Helper method to add a data line to the correct dictionary
def add_to_dict(input_dictionary, type_name):
    target_dict = globals()[('{}s'.format(type_name))]
    data_dict = globals()[('{}_data_dict'.format(type_name))]
    
    for key in data_dict.keys():
        target_dict[data_dict[key]].append(input_dictionary[key])
    
    return True

In [8]:
#Acceptable type_names
type_names = ['author','edition','work']

#Master function that can be modified for ease of use
#Runs the two helper functions for processing data
def process_data(input_string, type_line):
    # Generates the type_name from the type_line's format
    type_name = type_line.split('/')[-1:][0]
    
    if(type_name not in type_names):
        return
    
    try:
        verified_dict = verify_input(input_string, type_name)
        add_to_dict(verified_dict, type_name)
    except Exception as e:
        exceptions.append(('process data',e, input_string))

In [9]:
def type_verify(input_key, input_data, type_dict):
    target_type = type_dict[input_key]
    
    if(target_type == str):
        s = str(input_data)
        s = s.replace('\'','').replace('\"','')
        if s == '':
            s = '-1'
        s = str(s)
        s = '\''+s+'\''
        return s
    
    if(target_type == int):
        return int(input_data)
    
    if(target_type == list):
        r = 'ARRAY [$$NULL$$]'
        data = [str(s).replace('\'','').replace('\"','') for s in input_data]
        if (len(data) != 0):
            r = 'ARRAY {}'.format(data)
        return r
    
    return 'Error'

In [10]:
def generate_lines(input_dict, types_dict):
    lines = {}
    
    for counter in range(0,len(input_dict[list(input_dict.keys())[0]])-1):
        line = []
        for key in input_dict.keys():
            processed_val = type_verify(key, input_dict[key][counter], types_dict)
            line.append(processed_val)
        lines[counter] = line   
    return lines

In [11]:
def generate_query(input_lines, table_name, columns):
    qBase = 'INSERT INTO "{}" ({}) VALUES '.format(table_name, columns)
    
    lines = []
    
    for line,counter in zip(input_lines, range(0,len(input_lines))):
        data = [str(x) for x in input_lines[line]]
        try:
            lines.append('('+','.join(data)+')')
        except Exception as e:
            exceptions.append((e,input_lines[line],counter))
    
    if(len(lines) == 0):
        return ''
    
    qBase = qBase + ','.join(lines) + ';'
    
    return qBase

In [12]:
def send_data(query):
    
    if(query != ''):
        q(query)
        return True
    else:
        print('\t\tNo data')
        return False

In [14]:
def to_database(input_dict, type_name):
    # Select the data dict
    data_dict = globals()[('{}_data_dict'.format(type_name))]
    # Select the types dictionary
    type_table = globals()[('{}s_types_db'.format(type_name))]
    # Set Table Name
    table_name = type_name+'s'
    # generate the columns on the database side with proper formatting
    columns = str(list(data_dict.values())).replace('[','').replace(']','').replace('\'', '\"')
    
    print('\tformatting lines')
    # generate the lines in a dictionary of strings where line 0 is at {0:line_data}
    formatted_lines = generate_lines(input_dict, type_table)
    
    print('\tgenerating query')
    # generate the final query from the formatted lines, table name, and columns string
    final_query = generate_query(formatted_lines, table_name, columns)
    
    print('\tquerying')
    # query the database adding the data 
    send_data(final_query)
    
    return True

In [15]:
import pickle

In [16]:
last_position = pickle.load(open('last_pos.pkl','rb'))

In [17]:
import time as t

exceptions = []
counter = 0

authors = generate_dictionaries('author')
editions = generate_dictionaries('edition')
works = generate_dictionaries('work')

with open(r'D:\New folder (3)\ol_dump_2020-01-31.txt\ol_dump_2020-01-31.txt', 'r', encoding='utf-8') as f:
    f.seek(last_position)
    while(True):
        line = f.readline()
        if(line == ''):
            break
        
        #Split the line by tabs to identifiy important chunks
        line = line.split('\t')
        
        #Run the process_data function
        process_data(line[4], line[0].strip())
        
        #Add to the batch counter
        counter += 1
        
        #Every 50,000 lines processed, send it to database
        if(counter%50000 == 0):
            last_position = f.tell()
            pickle.dump(last_position, open('last_pos.pkl','wb'))
            print('Exporting: #{}'.format(str(counter//50000)))
            start = t.time()
            print('\t#Sending Authors#')
            to_database(authors, 'author')
            print('\t#Sending Editions#')
            to_database(editions, 'edition')
            print('\t#Sending Works#')
            to_database(works, 'work')
            del(authors)
            del(editions)
            del(works)
            authors = generate_dictionaries('author')
            editions = generate_dictionaries('edition')
            works = generate_dictionaries('work')
            total =  t.time()-start
            print('\tExport time (s): {}'.format(total))
            print('\tCounter: {}'.format(counter))

# Final export of leftover data
to_database(authors, 'author')
to_database(editions, 'edition')
to_database(works, 'work')

Exporting: #1
	#Sending Authors#
	formatting lines
	generating query
	querying
	#Sending Editions#
	formatting lines
	generating query
	querying
	#Sending Works#
	formatting lines
	generating query
	querying
	Export time (s): 5.599374294281006
	Counter: 50000
Exporting: #2
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	formatting lines
	generating query
	querying
	#Sending Works#
	formatting lines
	generating query
	querying
		No data
	Export time (s): 10.261246681213379
	Counter: 100000
Exporting: #3
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	formatting lines
	generating query
	querying
	#Sending Works#
	formatting lines
	generating query
	querying
	Export time (s): 9.73655390739441
	Counter: 150000
Exporting: #4
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	formatting lines
	generating query
	querying
		No data
	#Sending Works#
	forma

Traceback (most recent call last):
  File "<ipython-input-2-e92284ccb91b>", line 9, in q
    curs.execute(query)
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 3147817-3147818: surrogates not allowed


	Export time (s): 2.7699973583221436
	Counter: 1850000
Exporting: #38
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	formatting lines
	generating query
	querying
		No data
	#Sending Works#
	formatting lines
	generating query
	querying
	Export time (s): 7.7777276039123535
	Counter: 1900000
Exporting: #39
	#Sending Authors#
	formatting lines
	generating query
	querying
	#Sending Editions#
	formatting lines
	generating query
	querying
		No data
	#Sending Works#
	formatting lines
	generating query
	querying
	Export time (s): 4.2055816650390625
	Counter: 1950000
Exporting: #40
	#Sending Authors#
	formatting lines
	generating query
	querying
	#Sending Editions#
	formatting lines
	generating query
	querying
	#Sending Works#
	formatting lines
	generating query
	querying
		No data
	Export time (s): 10.812444686889648
	Counter: 2000000
Exporting: #41
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	fo

Traceback (most recent call last):
  File "<ipython-input-2-e92284ccb91b>", line 9, in q
    curs.execute(query)
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 231084-231085: surrogates not allowed


Exporting: #172
	#Sending Authors#
	formatting lines
	generating query
	querying
	#Sending Editions#
	formatting lines
	generating query
	querying
	#Sending Works#
	formatting lines
	generating query
	querying
		No data
	Export time (s): 6.980070352554321
	Counter: 8600000
Exporting: #173
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	formatting lines
	generating query
	querying
	#Sending Works#
	formatting lines
	generating query
	querying
		No data
	Export time (s): 9.963536977767944
	Counter: 8650000
Exporting: #174
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	formatting lines
	generating query
	querying
	#Sending Works#
	formatting lines
	generating query
	querying
	Export time (s): 9.544242143630981
	Counter: 8700000
Exporting: #175
	#Sending Authors#
	formatting lines
	generating query
	querying
		No data
	#Sending Editions#
	formatting lines
	generating query
	querying
		No data
	

True

In [23]:
last_position

16130758070

In [22]:
len(exceptions)

852

In [ ]:
globals()

In [54]:
import random

attributes = ['tall', 'short', 'big', 'small', 'long', 'dumb', 'smart', 'giraffey', 'giraffeless', 'left-handed', 'right-handed']

lists = []

for x in range(0,50000):
    att_list = []
    for y in range(0,10):
        att_list.append(random.choice(attributes))
        
    lists.append(att_list)

In [71]:
def join1(input_list):
    base = 'big giraffe is: '
    
    start = t.time()
    
    counter = 0
    for sub_list in input_list:
        base = base + '('+','.join(sub_list)+'), '
        counter += 1
        if (counter % 10000 == 0):
            print('Counter:{}, Run time (s): {}'.format(counter, t.time()-start))
     
    return base


def join2(input_list):
    base = 'big giraffe is: '
    
    start = t.time()
    processed = []
    
    counter = 0
    for sub_list in input_list:
        processed.append('('+','.join(sub_list)+'), ')
        counter += 1
        if (counter % 10000 == 0):
            print('Counter:{}, Run time (s): {}'.format(counter, t.time()-start))
        
    base = base + ''.join(processed)
    
    return base

In [47]:
import time as t

In [64]:
start = t.time()
results1 = join1(lists)
print('Run time (s): {}'.format(t.time()-start))

Counter:10000, Run time (s): 0.2890200614929199
Counter:20000, Run time (s): 6.906020402908325
Counter:30000, Run time (s): 21.157018899917603
Counter:40000, Run time (s): 40.78099799156189
Counter:50000, Run time (s): 66.7850193977356
Run time (s): 66.78598737716675


In [72]:
start = t.time()
results2 = join2(lists)
print('Run time (s): {}'.format(t.time()-start))

Counter:10000, Run time (s): 0.010001420974731445
Counter:20000, Run time (s): 0.019001483917236328
Counter:30000, Run time (s): 0.0279996395111084
Counter:40000, Run time (s): 0.03900146484375
Counter:50000, Run time (s): 0.05100059509277344
Run time (s): 0.07000064849853516


In [75]:
results1 == results2

True

In [21]:
data_query_w = """
    SELECT *
    FROM "works"
    WHERE "WorkTitle" ILIKE '%harry potter and the prisoner of azkaban%'
    LIMIT 10
"""

In [22]:
data_query_e = """
    SELECT *
    FROM "editions"
    WHERE "EditionTitle" ILIKE '%harry potter and the prisoner of azkaban%'
    LIMIT 10
"""

In [23]:
w = q(data_query_w, verbosity=True)
e = q(data_query_e, verbosity=True)

False

In [26]:
for x,y in zip(w,e):
    print(x)
    print(y)
    print()

('Harry Potter and the Prisoner of Azkaban', ['{type: {key: /type/author_role}, author: {key: /authors/OL23919A}}'], ['NULL'], '{type: /type/text, value: See https://openlibrary.org/works/OL13716956W/Harry_Potter_and_the_Prisoner_of_Azkaban}', '-1', ['NULL'], ['NULL'], ['NULL'], 1817637)
('Harry Potter and the Prisoner of Azkaban', ['NULL'], ['NULL'], ['NULL'], '2010 November', '[Bloomsbury]', 0, 0, 0, ['NULL'], ['NULL'], '-1', ['NULL'], ['NULL'], ['{key: /works/OL13716956W}'], ['{key: /languages/eng}'], '[9781408810569]', 638458)

('Harry Potter and the Prisoner of Azkaban', ['{type: {key: /type/author_role}, author: {key: /authors/OL23919A}}'], ['NULL'], '{type: /type/text, value: https://openlibrary.org/works/OL13716956W/Harry_Potter_and_the_Prisoner_of_Azkaban}', '-1', ['NULL'], ['NULL'], ['NULL'], 1972925)
('Harry Potter and the Prisoner of Azkaban - Hufflepuff Edition', ['{key: /authors/OL23919A}'], ['NULL'], ['NULL'], '2019', '[Bloomsbury]', 0, 0, 0, ['NULL'], ['NULL'], '-1', ['

In [38]:
import requests

In [39]:
s = 'https://www.googleapis.com/books/v1/volumes?q=flowers+inauthor'

r = requests.request('GET', s)

In [40]:
r

<Response [200]>

In [42]:
r.json()

{'kind': 'books#volumes',
 'totalItems': 3288,
 'items': [{'kind': 'books#volume',
   'id': 'cRYjgfymePYC',
   'etag': 'DLan+WG7zdU',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/cRYjgfymePYC',
   'volumeInfo': {'title': 'The War Of The Flowers',
    'authors': ['Tad Williams'],
    'publisher': 'Penguin',
    'publishedDate': '2006-09-05',
    'description': "Theo Vilmos' life is about to take a real turn for the worse. He is drawn from his home in Northern California into the parallel world of Faerie, for, unknown to him, he is a pivotal figure in a war between certain of Faerie's powerful lords and the rest of the strange creatures who live in this exotic realm.",
    'industryIdentifiers': [{'type': 'ISBN_13', 'identifier': '9781101657645'},
     {'type': 'ISBN_10', 'identifier': '1101657642'}],
    'readingModes': {'text': True, 'image': False},
    'pageCount': 832,
    'printType': 'BOOK',
    'categories': ['Fiction'],
    'averageRating': 4.0,
    'ratingsCount'